In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import time
import random
import os
from datetime import datetime, timedelta
import winsound



marca = 'jeep'
modelos = ['wrangler']  

base_url = "https://autos.mercadolibre.com.ar/{marca}/{modelo}/_Desde_{offset}_ITEM*CONDITION_2230581_NoIndex_True"

user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; rv:102.0) Gecko/20100101 Firefox/102.0',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15',
]


def extraer_datos_auto(url_auto, nombre_csv):
    try:
        headers = {'User-Agent': random.choice(user_agents)}
        response_auto = requests.get(url_auto, headers=headers)
        if response_auto.status_code == 200:
            soup_auto = BeautifulSoup(response_auto.text, 'html.parser')

            info_tag = soup_auto.find('span', class_='ui-pdp-subtitle')
            if info_tag:
                info_text = info_tag.text.strip()
                partes = info_text.split('|')
                if len(partes) > 1:
                    año = partes[0].strip()
                    km_text = partes[1].split('·')[0].strip()

                    if año.isdigit() and int(año) < 2025:
                        script_tag = soup_auto.find('script', type='application/ld+json')
                        if script_tag:
                            json_content = script_tag.string
                            data = json.loads(json_content)

                            # Extraer la fecha de publicación exactamente como aparece en el HTML
                            fecha_publicacion = "No disponible"
                            info_subtitle = soup_auto.find('span', class_='ui-pdp-subtitle')
                            if info_subtitle:
                                fecha_publicacion = info_subtitle.text.strip().split('·')[-1].strip()  # Extraer "Publicado hace X días"

                            fila = {
                                'Marca': data.get('brand', 'No disponible'),
                                'Modelo': data.get('name', 'No disponible').split()[1] if len(data.get('name', '').split()) > 1 else "No disponible",
                                'Año': año,
                                'Versión': ' '.join(data.get('name', '').split()[2:]) if len(data.get('name', '').split()) > 2 else "No disponible",
                                'Color': data.get('color', 'No disponible'),
                                'Tipo de combustible': data.get('fuelType', 'No disponible'),
                                'Puertas': data.get('numberOfDoors', 'No disponible'),
                                'Transmisión': data.get('vehicleTransmission', 'No disponible'),
                                'Motor': data.get('vehicleEngine', 'No disponible'),
                                'Tipo de carrocería': data.get('bodyType', 'No disponible'),
                                'Fecha publicación': fecha_publicacion,
                                'Kilómetros': km_text,
                                'Precio': data.get('offers', {}).get('price', 'No disponible')                                
                            }

                            df = pd.DataFrame([fila])
                            modo = 'a' if os.path.exists(nombre_csv) else 'w'
                            encabezado = not os.path.exists(nombre_csv)

                            df.to_csv(nombre_csv, index=False, mode=modo, header=encabezado, encoding='utf-8')
                            return True
        else:
            print(f"Error al acceder a la página del auto. Código: {response_auto.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"Error de solicitud: {e}")
    return False

def scrapear_pagina(url, nombre_csv):
    try:
        headers = {'User-Agent': random.choice(user_agents)}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            enlaces_autos = soup.find_all('a', href=True)

            autos_encontrados = 0
            for enlace in enlaces_autos:
                href = enlace['href']
                if href.startswith("https://auto.mercadolibre.com.ar/MLA-"):
                    if extraer_datos_auto(href, nombre_csv):
                        autos_encontrados += 1
                    time.sleep(random.uniform(0.1, 0.2))

            if autos_encontrados == 0:
                print("No se encontraron más autos en esta página.")
                return False
            return True
        else:
            print(f"Error al acceder a la página. Código: {response.status_code}")
            return False
    except requests.exceptions.RequestException as e:
        print(f"Error de solicitud: {e}")
        return False

for modelo in modelos:
    print(f"Scrapeando modelo: {modelo}")
    nombre_csv = f'autos_{marca}_{modelo}_mercadolibre.csv'
    autos_scrapeados = 0
    offset = 0

    if os.path.exists(nombre_csv):
        df_existente = pd.read_csv(nombre_csv)
        autos_scrapeados = len(df_existente)
        offset = (autos_scrapeados // 48) * 48
        print(f"Archivo existente detectado para {modelo}. Retomando desde offset {offset}...")
    else:
        print(f"Comenzando nuevo archivo para {modelo}...")

    while True:
        url_pagina = base_url.format(marca=marca, modelo=modelo, offset=offset)
        print(f"Scrapeando página con offset {offset}...")
        if not scrapear_pagina(url_pagina, nombre_csv):
            break
        offset += 48
        time.sleep(random.uniform(0.1, 0.2))

    if os.path.exists(nombre_csv):
        autos_scrapeados = len(pd.read_csv(nombre_csv))

    print(f"Finalizado el modelo {modelo}. Total autos guardados: {autos_scrapeados}\n")
winsound.Beep(1000, 1000)  # frecuencia 1000 Hz, duración 500 ms


Scrapeando modelo: wrangler
Comenzando nuevo archivo para wrangler...
Scrapeando página con offset 0...
Scrapeando página con offset 48...
Scrapeando página con offset 96...
Scrapeando página con offset 144...
No se encontraron más autos en esta página.
Finalizado el modelo wrangler. Total autos guardados: 142

